In [1]:
import os
sep_local = os.path.sep
sep_local

'\\'

In [2]:
%env TF_KERAS = 1

env: TF_KERAS=1


In [3]:
import sys
sys.path.append('..'+sep_local+'..')

In [4]:
#done in case of linux
#import os
#os.chdir("/home/azeghost/git/Generative_Models")

In [5]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.0.0-rc1'

In [8]:
dataset_name = 'dsprites'

In [9]:
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='dsprites',
    version=0.1.0,
    description='dSprites is a dataset of 2D shapes procedurally generated from 6 ground truth
independent latent factors. These factors are *color*, *shape*, *scale*,
*rotation*, *x* and *y* positions of a sprite.

All possible combinations of these latents are present exactly once,
generating N = 737280 total images.

### Latent factor values

*   Color: white
*   Shape: square, ellipse, heart
*   Scale: 6 values linearly spaced in [0.5, 1]
*   Orientation: 40 values in [0, 2 pi]
*   Position X: 32 values in [0, 1]
*   Position Y: 32 values in [0, 1]

We varied one latent at a time (starting from Position Y, then Position X, etc),
and sequentially stored the images in fixed order.
Hence the order along the first dimension is fixed and allows you to map back to
the value of the latents corresponding to that image.

We chose the latents values deliberately to have the smallest step changes
while ensuring that all pixel outp

In [10]:
intermediate_dim = 20
inputs_shape=(64, 64, 1) # image shape
batch_size = 100
latent_dim = 6

In [11]:
TRAIN_BUF = 600
TEST_BUF = 100


In [12]:
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).batch(batch_size)

In [13]:
# import tensorflow_datasets as tfds
# # Construct a tf.data.Dataset
# train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
# try:
#     test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
# except:
#     test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)


In [14]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'][0].numpy().max())
    break

In [15]:
_instance_scale

1.0

In [16]:
enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim, dtype='float32')
]

dec_lays2 = [
    tf.keras.layers.Dense(units=16*16*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(16, 16, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME", dtype='float32')
]

In [17]:
enc_mean_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

enc_var_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

dec_lays = [tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu')]

## First Time

In [18]:
#from utils.data_and_files.file_utils import make_random_string
#from time import gmtime, strftime

#model_name = 'AE_' + make_random_string(5) + strftime("%a_%d_%b_%Y_%H_%M", gmtime())
#print(model_name)

In [19]:
model_name = dataset_name+'AE'

In [20]:
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [21]:
import os

In [22]:
os.path.join(recoding_dir, model_name)

'..\\..\\recoding\\dspritesAE\\dspritesAE'

In [23]:
pwd

'C:\\projects\\Generative_Models\\notebooks\\Dstripes'

In [24]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

In [25]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays2#enc_mean_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays2#dec_lays
    }
]

In [26]:
from utils.data_and_files.file_utils import create_if_not_exist

In [27]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [28]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\dspritesAE\\var_save_dir'

In [29]:
#to restore trained model, set filepath=_restore

In [30]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 14400)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 86406     
_________________________________________________________________
batch_normalization (BatchNo (None, 6)                 24        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
activity_regularization (Act (None, 6)                 0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              57344     
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 32)        18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 1)         289       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 1)         4         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 1)         0

  WARNING  | None


In [31]:
ae.compile(metrics=None)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "dspritesAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 64, 64, 1)]       0         
_________________________________________________________________
inference (Sequential)       (None, 6)                 105246    
_________________________________________________________________
generative (Sequential)      (None, 64, 64, 1)         94597     
_________________________________________________________________
x_logits (TensorFlowOpLayer) [(None, 64, 64, 1)]       0         
Total params: 199,843
Trainable params: 199,829
Non-trainable params: 14
_________________________________________________________________
None


In [32]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

Using TensorFlow backend.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [33]:
progbar = NotebookPrograssBar(leave_outer=False)

In [34]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [35]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [36]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\dspritesAE\\csv_dir\\dspritesAE.csv'

In [37]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [38]:
sg = SampleGeneration(latent_shape=6, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [39]:
def cast_batch(batch):
    return tf.cast(batch['image'], dtype=tf.float32), tf.cast(batch['image'], dtype=tf.float32)
ds = train_ds.map(cast_batch)

In [40]:
def rep_fn(x):
    return ae.encode(x)

In [41]:
from training.callbacks.gt_metrics import GTMetrics

In [42]:
from data.gt_load.datasets import load
DATA_PATH = '..'+sep_local+'..'+sep_local+'data'+sep_local+'.gt_datasets'

In [43]:
eval_dataset = load(dataset_name='dsprites_full', dataset_path=DATA_PATH)

In [44]:
gt_csv = os.path.join(recoding_dir, 'csv_dir', 'gt_metrics')

In [45]:
import numpy as np

In [46]:
gt_mertics = GTMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=rep_fn,
    random_state=np.random.RandomState(0),
    file_Name=gt_csv,
    num_train=1000,
    num_test=200,
    batch_size=32,
    continuous_factors=False,
    gt_freq=2
)

In [ ]:
ae.fit_generator(
    generator=ds,
    steps_per_epoch=50,
    epochs=100, 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg, gt_mertics],
    workers=-1,
    use_multiprocessing=True
)

In [ ]:
# import os

# # # TF 1.x
# # resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# # tf.tpu.experimental.initialize_tpu_system(resolver)
# # strategy = tf.distribute.experimental.TPUStrategy(resolver)

# # TF 2.0
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_host(resolver.master())
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

# with strategy.scope():
#     #create 
#     #compile

# #fit    